# Transferring Senator Data From ProPublica to SQLite DB

In [1]:
import configparser
import requests
import sqlite3

In [2]:
# Config API
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config.get('propublica', 'PROPUBLICA_API_KEY')

In [3]:
# Instantiate SQLite DB
conn = sqlite3.connect('cong_80_116.db')

In [4]:
# Determine columns for database (Note: 101st Congress and above has 45 columns, 3 are constructed)
r = requests.get(
        f'https://api.propublica.org/congress/v1/80/senate/members.json',
        headers={'X-API-Key': api_key}
)
sample = r.json()['results'][0]['members'][0]
cols = list(sample.keys())
print(f'First 5 columns: {cols[:5]}')
print(f'Number of columns: {len(cols)}')

First 5 columns: ['id', 'title', 'short_title', 'api_uri', 'first_name']
Number of columns: 42


In [5]:
# Add congress number column and format for 
cols = [ col + ' TEXT' for col in cols ]
cols = ', '.join(cols)
cols = cols + ', congress TEXT'

In [6]:
# Create senator table
c = conn.cursor()
c.execute(
    f'''
    CREATE TABLE senators
    ({cols})
    ;
    '''
)
c.close()

In [7]:
# Check columns
c = conn.cursor()
c.execute(
    '''
    PRAGMA table_info(senators)
    ;
    '''
)
print(c.fetchall()[-5:])
c.close()

[(38, 'state', 'TEXT', 0, None, 0), (39, 'senate_class', 'TEXT', 0, None, 0), (40, 'state_rank', 'TEXT', 0, None, 0), (41, 'lis_id', 'TEXT', 0, None, 0), (42, 'congress', 'TEXT', 0, None, 0)]


In [8]:
# Function to get data from ProPublica API
def get_congress(n, chamber):
    r = requests.get(
        f'https://api.propublica.org/congress/v1/{n}/{chamber}/members.json',
        headers={'X-API-Key': api_key}
    )
    results = r.json()['results'][0]['members']
    return results

# Function to extract congressional members
def get_members(n, congress):
    if len(congress[0]) == 43:
        members = [ tuple(list(member.values()) + [f'{n}']) for member in congress ]
    else:
        members = [ tuple(list(member.values())[:42] + [f'{n}']) for member in congress ]
    return members

# Function to insert members
def insert_members(n, chamber):
    # SQL insert statement
    s = ','.join('?'*43) # 43 value columns
    insert_statement = f'''INSERT INTO senators VALUES ({s})'''
    
    congress = get_congress(n, chamber)
    members = get_members(n, congress)
    c = conn.cursor()
    c.executemany(insert_statement, members)
    c.close()

In [9]:
# Set parameters
congresses = range(80, 117) # from the 80th congress to the 116th congress

# Insert members
for n in congresses:
    insert_members(n)